CREACIÓN DATASET MODELO 2 V1

In [1]:
# Import librerias 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Rutas de archivos 

# Ruta donde se encuentran los archivos CSV
ruta_datos_mimic = "C:/Users/BiDAlab/TFG_Alfonso/BBDD/mimic-iv-3.1/mimic-iv-3.1/" 

# Importar archivos CSV específicos
admissions = pd.read_csv(ruta_datos_mimic + "hosp/admissions.csv/admissions.csv")
diagnoses = pd.read_csv(ruta_datos_mimic + "hosp/diagnoses_icd.csv/diagnoses_icd.csv")
icu_stays = pd.read_csv(ruta_datos_mimic + "icu/icustays.csv/icustays.csv")

# Ruta guardado
ruta_guardado = "C:/Users/BiDAlab/TFG_Alfonso/Pruebas/Modelo2/MIMIC/CSV/"

In [22]:
# Buscamos en el fichero chartevents (sucesos), los que nos interesan, a través de IDsabs

# 220050 -> presion sistólica
# 220051 -> presión diastólica
# 220052 -> presión mean ([sistólica + 2*diastólica]/3))

# 223761 -> temp. corporal Fº
# 223762 -> temp. corporal Cº
itemids_interes = [223761, 223762]  # IDs de ejemplo
chunksize = 1000000

filtered_data = []

for chunk in pd.read_csv(ruta_datos_mimic + "icu/chartevents.csv/chartevents.csv", chunksize=chunksize):
    chunk_filtered = chunk[chunk['itemid'].isin(itemids_interes)]
    filtered_data.append(chunk_filtered)

chartevents = pd.concat(filtered_data)

# Mensaje finalización
print(f"Se ha finalizado la lectura de archivos\n")

Se ha finalizado la lectura de archivos



In [23]:
# Guardamos los datos extraidos de chartevents

chartevents.to_csv(ruta_guardado +"chartevents_temperatura.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_temperatura.csv'")

Resultados guardados en 'CSV/chartevents_temperatura.csv'


In [24]:
# Filtramos los pacientes con sepsis con los códigos ICD

# ICD9 : 99591 -> sepsis
#        99592 -> sepsis severa
#        78552 -> septic shock
# ICD10 : A419 -> sepsis no especificada (sangre infectada confirmada pero causa sin especificar)
#         R6521 -> sepsis severa
#         R6520 -> septic shock
codigos_sepsis = ['99591', '99592', '78552', 'R6521', 'A419', 'R6520']

# Filtrar diagnósticos por códigos de sepsis
sepsis_cases = diagnoses[diagnoses['icd_code'].isin(codigos_sepsis)]

# Combinar diagnósticos de sepsis con admisiones para obtener detalles
sepsis_details = pd.merge(sepsis_cases, admissions, on='subject_id', how='inner')

# Mostrar los casos de sepsis
print("Número de casos de sepsis:", sepsis_details['subject_id'].nunique())
print(sepsis_details.head())

Número de casos de sepsis: 15880
   subject_id  hadm_id_x  seq_num icd_code  icd_version  hadm_id_y  \
0    10000826   21086876        2    99591            9   20032235   
1    10000826   21086876        2    99591            9   21086876   
2    10000826   21086876        2    99591            9   28289260   
3    10001401   27012892        3    R6520           10   20144849   
4    10001401   27012892        3    R6520           10   21544441   

             admittime            dischtime deathtime  \
0  2146-12-05 19:07:00  2146-12-12 16:30:00       NaN   
1  2146-12-18 17:39:00  2146-12-24 19:55:00       NaN   
2  2146-12-31 00:43:00  2147-01-02 17:45:00       NaN   
3  2136-11-20 14:20:00  2136-11-23 14:00:00       NaN   
4  2131-06-04 00:00:00  2131-06-15 16:10:00       NaN   

                admission_type admit_provider_id  admission_location  \
0                     EW EMER.            P278S6      EMERGENCY ROOM   
1                 DIRECT EMER.            P61YAW     CLINIC

In [25]:
# Obtenemos una lista con los pacientes unicos con sepsis

n_pacientes_unicos_sepsis = sepsis_details['subject_id'].unique()
print(n_pacientes_unicos_sepsis)
print(f"Número de pacientes únicos: {sepsis_details['subject_id'].nunique()}")

[10000826 10001401 10001843 ... 19995732 19997367 19997886]
Número de pacientes únicos: 15880


In [26]:
# Filtramos chartevents para quedarnos con los sucesos de los casos con ssepsis_cases

chartevents_sepsis = chartevents[chartevents['subject_id'].isin(n_pacientes_unicos_sepsis)]

# Filtrado por pacientes únicos y múltiples itemid
chartevents_sepsis = chartevents[
    (chartevents['subject_id'].isin(n_pacientes_unicos_sepsis))]

# Guardar los detalles de sepsis en un archivo CSV
chartevents_sepsis.to_csv(ruta_guardado +"chartevents_sepsis_temperatura.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_sepsis_temperatura.csv'")

Resultados guardados en 'CSV/chartevents_sepsis_temperatura.csv'


In [27]:
# Pasamos todos los datos a grados celsius

chartevents_sepsis = pd.read_csv(ruta_guardado + "chartevents_sepsis_temperatura.csv")

# Creo una nueva columna, donde si la temperatura se ha tomado en valuenum en Fº se pase a Cº y se copie en la nueva col.
# Convertir temperaturas de °F a °C, y crear la nueva columna "temperature_C"
chartevents_sepsis.loc[:, "temperature_C"] = np.where(
    chartevents_sepsis["valueuom"] == "°F",
    ((chartevents_sepsis["valuenum"] - 32) * 5/9).round(1),
    chartevents_sepsis["valuenum"].round(1)
)

# Verifica las primeras filas para comprobar la conversión
print(chartevents_sepsis[["valuenum", "valueuom", "temperature_C"]].head(10))

# Elimina la columna "valueuom" si existe
if "valueuom" in chartevents_sepsis.columns:
    chartevents_sepsis.drop(columns=["valueuom"], inplace=True)

# Reemplaza "valuenum" por los valores convertidos a Celsius
chartevents_sepsis["valuenum"] = chartevents_sepsis["temperature_C"]

# Elimina la columna temporal "temperature_C"
chartevents_sepsis.drop(columns=["temperature_C", "value", "caregiver_id", "storetime"], inplace=True)

# Cambio todos los item_id como medida de temperatura en celsius
chartevents_sepsis["itemid"] = 223762

# Muestra el resultado final
print(chartevents_sepsis.head(10))


   valuenum valueuom  temperature_C
0      97.6       °F           36.4
1      97.5       °F           36.4
2      97.9       °F           36.6
3      37.3       °C           37.3
4      37.4       °C           37.4
5      37.5       °C           37.5
6      37.5       °C           37.5
7      37.5       °C           37.5
8      37.4       °C           37.4
9      37.6       °C           37.6
   subject_id   hadm_id   stay_id            charttime  itemid  valuenum  \
0    10001843  26133978  39698942  2134-12-06 00:00:00  223762      36.4   
1    10001843  26133978  39698942  2134-12-06 04:00:00  223762      36.4   
2    10001843  26133978  39698942  2134-12-05 20:00:00  223762      36.6   
3    10002013  23581541  39060235  2160-05-18 20:00:00  223762      37.3   
4    10002013  23581541  39060235  2160-05-18 21:00:00  223762      37.4   
5    10002013  23581541  39060235  2160-05-18 22:00:00  223762      37.5   
6    10002013  23581541  39060235  2160-05-18 23:00:00  223762      37.5

In [28]:
# Guardar los detalles de sepsis en un archivo CSV
chartevents_sepsis.to_csv(ruta_guardado +"chartevents_sepsis_temperatura_celsius.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_sepsis_temperatura_celsius.csv'")

Resultados guardados en 'CSV/chartevents_sepsis_temperatura_celsius.csv'


In [29]:
archivo_combinado = pd.read_csv("C:/Users/BiDAlab/TFG_Alfonso/Pruebas/Modelo1/MIMIC_csv/archivo_combinado_ordenado_MIMIC.csv")

# Agrupar por subject_id y contar el número total de warnings
warnings_por_paciente = chartevents_sepsis.groupby("subject_id")["warning"].sum().reset_index()

# Renombrar la columna para mayor claridad
warnings_por_paciente.rename(columns={"warning": "total_warnings_temperatura"}, inplace=True)


archivo_combinado_final = pd.merge(archivo_combinado, warnings_por_paciente, on='subject_id', how='inner')

In [30]:
# Esto solo se aplica a los casos de sepsis, hay que hacer los casos de sepsis y no sepsis por separado

print(archivo_combinado_final.head())

   subject_id  gender  anchor_age  sis_desviacion  dias_desviacion  \
0    10002013       0          53       16.689746         8.576824   
1    10002428       0          80       16.925170        12.605705   
2    10003400       0          72       10.814669         9.766610   
3    10003637       1          57       16.366693         6.206981   
4    10004235       1          47       14.743613        10.639475   

   mean_desviacion   sis_media  dias_media  mean_media  sis_mediana  \
0        13.514515  110.619048   61.476190   76.700000        116.0   
1        15.075749  110.987220   59.006390   79.524845        109.0   
2         9.163913  102.727749   64.884817   78.518325        102.0   
3         7.390996  123.716667   54.166667   71.808333        124.5   
4        11.361041  108.368421   68.131579   79.815789        104.5   

   dias_mediana  mean_mediana  dias_tendencia  sis_tendencia  mean_tendencia  \
0          60.0          75.5   -2.027962e-04  -2.373182e-04   -2.865596

In [31]:
# Ahora hago una lista con los pacientes sin sepsis

subject_ids_sepsis_0 = archivo_combinado[archivo_combinado['sepsis'] == 0]['subject_id'].tolist()


In [32]:
# Miro en chatevents su temperatura

# Filtramos chartevents para quedarnos con los sucesos de los casos SIN sepsis

chartevents_no_sepsis = chartevents[chartevents['subject_id'].isin(subject_ids_sepsis_0)]

# Guardar los detalles de sepsis en un archivo CSV
chartevents_no_sepsis.to_csv(ruta_guardado +"chartevents_NO_sepsis_temperatura.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_NO_sepsis_temperatura.csv'")

Resultados guardados en 'CSV/chartevents_NO_sepsis_temperatura.csv'


In [33]:
# Pasamos todos los datos a grados celsius

chartevents_no_sepsis = pd.read_csv(ruta_guardado + "chartevents_NO_sepsis_temperatura.csv")

# Creo una nueva columna, donde si la temperatura se ha tomado en valuenum en Fº se pase a Cº y se copie en la nueva col.
# Convertir temperaturas de °F a °C, y crear la nueva columna "temperature_C"
chartevents_no_sepsis.loc[:, "temperature_C"] = np.where(
    chartevents_no_sepsis["valueuom"] == "°F",
    ((chartevents_no_sepsis["valuenum"] - 32) * 5/9).round(1),
    chartevents_no_sepsis["valuenum"].round(1)
)

# Verifica las primeras filas para comprobar la conversión
print(chartevents_no_sepsis[["valuenum", "valueuom", "temperature_C"]].head(10))

# Elimina la columna "valueuom" si existe
if "valueuom" in chartevents_no_sepsis.columns:
    chartevents_no_sepsis.drop(columns=["valueuom"], inplace=True)

# Reemplaza "valuenum" por los valores convertidos a Celsius
chartevents_no_sepsis["valuenum"] = chartevents_no_sepsis["temperature_C"]

# Elimina la columna temporal "temperature_C"
chartevents_no_sepsis.drop(columns=["temperature_C", "value", "caregiver_id", "storetime"], inplace=True)

# Cambio todos los item_id como medida de temperatura en celsius
chartevents_no_sepsis["itemid"] = 223762

# Muestra el resultado final
print(chartevents_no_sepsis.head(10))


   valuenum valueuom  temperature_C
0      97.7       °F           36.5
1      96.2       °F           35.7
2      95.6       °F           35.3
3      95.4       °F           35.2
4      96.8       °F           36.0
5      95.5       °F           35.3
6      95.5       °F           35.3
7      96.5       °F           35.8
8      96.4       °F           35.8
9      94.0       °F           34.4
   subject_id   hadm_id   stay_id            charttime  itemid  valuenum  \
0    10002155  20345487  32358465  2131-03-09 21:58:00  223762      36.5   
1    10002155  20345487  32358465  2131-03-10 00:00:00  223762      35.7   
2    10002155  20345487  32358465  2131-03-10 01:00:00  223762      35.3   
3    10002155  20345487  32358465  2131-03-10 02:00:00  223762      35.2   
4    10002155  20345487  32358465  2131-03-10 03:00:00  223762      36.0   
5    10002155  20345487  32358465  2131-03-10 05:00:00  223762      35.3   
6    10002155  20345487  32358465  2131-03-10 05:30:00  223762      35.3

In [34]:
# Agrupar por subject_id y contar el número total de warnings
warnings_por_paciente = chartevents_no_sepsis.groupby("subject_id")["warning"].sum().reset_index()

# Renombrar la columna para mayor claridad
warnings_por_paciente.rename(columns={"warning": "total_warnings_temperatura"}, inplace=True)

archivo_combinado_final_2 = pd.merge(archivo_combinado, warnings_por_paciente, on='subject_id', how='inner')

archivo_final_final = pd.concat([archivo_combinado_final, archivo_combinado_final_2])

# Ordenar por subject_id de menor a mayor
archivo_final_final = archivo_final_final.sort_values(by='subject_id').reset_index(drop=True)

# Guardar los detalles en un archivo CSV
archivo_final_final.to_csv(ruta_guardado +"archivo_combiando_temperatura.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/archivo_combiando_temperatura.csv'")

Resultados guardados en 'CSV/archivo_combiando_temperatura.csv'
